In [35]:
from datetime import datetime
from utils import utils


start_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 23, 0, 0))
end_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 26, 0, 0))

In [36]:
# save kucoin price weekly data
import requests


kucoin_url = "https://min-api.cryptocompare.com/data/v2/histominute?fsym=ETH&tsym=USD&limit=1000&toTs={}"

current_timestamp = end_timestamp
# current_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 19, 1, 32))
kucoin_result = []
while current_timestamp >= start_timestamp:
    print("Start processing, current time: ", utils.utc_timestamp_to_date(current_timestamp))
    response = requests.get(kucoin_url.format(current_timestamp))
    kucoin_data = response.json()['Data']
    if not kucoin_data:
        break
    for d in kucoin_data['Data'][::-1]:
        if d["time"] >= start_timestamp:
            kucoin_result.append(d)
        else:
            break
    current_timestamp = kucoin_data['TimeFrom'] - 60

kucoin_result = kucoin_result[::-1]

print(len(kucoin_result))

Start processing, current time:  2023-05-26 00:00:00+00:00
Start processing, current time:  2023-05-25 07:19:00+00:00
Start processing, current time:  2023-05-24 14:38:00+00:00
Start processing, current time:  2023-05-23 21:57:00+00:00
Start processing, current time:  2023-05-23 05:16:00+00:00
4321


In [19]:
print(kucoin_result[0])

{'time': 1684460940, 'high': 1798.38, 'low': 1797.75, 'open': 1797.75, 'volumefrom': 57.77, 'volumeto': 103880.66, 'close': 1798.05, 'conversionType': 'direct', 'conversionSymbol': ''}


In [22]:
# ARB graphQL
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport



arb_query = """
query {
    swaps(orderBy: timestamp, orderDirection: asc,
        where: { pool: "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443", timestamp_gte: %d }){
            id
            amountIn
            amountInUSD
            amountOut
            amountOutUSD
            tick
            timestamp
            tokenIn {
                symbol
                decimals
            }
            tokenOut {
                symbol
                decimals
            }
    }
}
"""


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

# # variables = { "num_skip": 0, "start_timestamp": 1684460940 }
# # response = client.execute(gql(arb_query), variable_values=variables)
response = client.execute(gql(arb_query % start_timestamp))
print(len(response["swaps"]))

100


In [23]:
print(response["swaps"][0])

# temp
result = []
id_set = set()

{'id': '0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea3c17e3a123540b281eca0b000000', 'amountIn': '18509968', 'amountInUSD': '18.509968', 'amountOut': '10174988585088354', 'amountOutUSD': '18.50071257554249565419706', 'tick': '-201264', 'timestamp': '1684800030', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}


In [38]:
print(arb_query % current_timestamp)


query {
    swaps(orderBy: timestamp, orderDirection: asc,
        where: { pool: "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443", timestamp_gte: 1684758900 }){
            id
            amountIn
            amountInUSD
            amountOut
            amountOutUSD
            tick
            timestamp
            tokenIn {
                symbol
                decimals
            }
            tokenOut {
                symbol
                decimals
            }
    }
}



In [39]:
# response = client.execute(gql(arb_query % start_timestamp))
# result = response['swaps']
# id_set = set()
#
# for item in result:
#     id_set.add(item["id"])

while True:
    print(datetime.now(), len(result), len(id_set))
    print("Current time: ", utils.utc_timestamp_to_date(int(result[-1]["timestamp"])))
    current_timestamp = int(result[-1]["timestamp"])
    query = arb_query % current_timestamp
    try:
        response = client.execute(gql(query))

        swap_data = response["swaps"]
        if len(swap_data) == 0:
            break
        for item in swap_data:
            if int(item["timestamp"]) > end_timestamp:
                break
            elif item["id"] in id_set:
                continue
            else:
                result.append(item)
                id_set.add(item["id"])
    except Exception as ex:
        print("Fetching swap data error", ex)

2023-05-26 18:34:28.026979 23386 23386
Current time:  2023-05-24 13:52:26+00:00


KeyboardInterrupt: 

In [37]:
# save data
import pandas as pd

kucoin_df = pd.DataFrame(kucoin_result)
kucoin_df.to_csv("../../data/kucoin_eth_price.csv", index=False)

In [34]:
import pandas as pd
from copy import deepcopy


arb_result = deepcopy(result)
for item in arb_result:
    item["tokenIn"] = item["tokenIn"]["symbol"]
    item["tokenOut"] = item["tokenOut"]["symbol"]

arb_uniswap_df = pd.DataFrame(arb_result)
arb_uniswap_df.to_csv("../../data/arb_swap.csv", index=False)

In [3]:
from decimal import Decimal

def sqrtpricex96_to_price(sqrtpricex96):
    """Converts a Uniswap SQRTpriceX96 to price.

    Args:
      sqrtpricex96: The SQRTpriceX96 to convert.

    Returns:
      The price.
    """

    return Decimal(sqrtpricex96) ** 2 / (2 ** 96)


print(sqrtpricex96_to_price(1874659030893028805438655155008306))
1874679522069631707331686660801156

4.435728875418589428481139256E+37


In [4]:
print(2 ** 96)

79228162514264337593543950336


In [25]:
1 / ((1874659030893028805438655155008306 / (1 << 96)) ** 2 / (10 ** 12))

1786.1362752201069

In [29]:
import math

def sqrtpricex96_to_price_eth_usd(sqrtpricex96):
    """Converts a Uniswap SQRTpriceX96 to price.

    Args:
      sqrtpricex96: The SQRTpriceX96 to convert.

    Returns:
      The price.
    """
    price = sqrtpricex96 / math.pow(2, 96)
    return 1 / (price ** 2 / (10 ** 12))


print(sqrtpricex96_to_price_eth_usd(1874659030893028805438655155008306))

1786.1362752201069


In [20]:
Decimal(1974045567390486984838358761822072) / (1 << 64)

Decimal('107013224637507.3118500159524')

In [31]:
TICK_BASE = 1.0001


def tick_to_price_eth_usd(tick):
    return 1 / (TICK_BASE ** tick / (10 ** 12))


print(tick_to_price_eth_usd(201442))

1786.1687802452334


In [33]:
print(tick_to_price_eth_usd(201438))

1786.883354934603


In [36]:
def arb_tick_to_price_eth_usd(tick):
    return TICK_BASE ** tick * 10 ** 12

print(arb_tick_to_price_eth_usd(-201439))

1786.7046844661563
